In [20]:
import mlrun
import os

In [8]:
url='git://<GIT_USERNAME>:<GIT_TOKEN>@github.com/GiladShapira94/remote-project.git'
project_name = 'nyc-remote-target'

In [10]:
project = mlrun.load_project(url=url,context='./target',name=project_name,init_git=True,user_project=True)

> 2022-05-25 14:29:12,947 [info] Username was normalized to match the required pattern for project name: {'username': 'Gilad', 'normalized_username': 'gilad'}


In [33]:
project = mlrun.load_project(context='./target',name=project_name,init_git=True,user_project=True)

> 2022-05-25 14:45:23,277 [info] Username was normalized to match the required pattern for project name: {'username': 'Gilad', 'normalized_username': 'gilad'}


# Set secrets

### For local execution 

In [21]:
os.environ['AWS_ACCESS_KEY_ID']='<AWS_ACCESS_KEY_ID>'
os.environ['AWS_SECRET_ACCESS_KEY']='<AWS_SECRET_ACCESS_KEY>'

In [22]:
os.environ['GIT_USERNAME']='<GIT_USERNAME>'
os.environ['GIT_PASSWORD']='<GIT_TOKEN>'

### For Remote execution - Saved as project/k8s secrets

In [23]:

# Using project-secrets (recommended) - project secrets are automatically mounted to project functions
secrets = {'AWS_ACCESS_KEY_ID': '<AWS_ACCESS_KEY_ID>', 'AWS_SECRET_ACCESS_KEY': '<AWS_SECRET_ACCESS_KEY>' 
           ,'GIT_USERNAME':'<GIT_USERNAME>','GIT_PASSWORD':'<GIT_TOKEN>'}
mlrun.get_run_db().create_project_secrets(
    project.name,
    provider=mlrun.api.schemas.SecretProviderName.kubernetes,
    secrets=secrets
)



# Run Functions

### Simple Job

In [14]:
taxi_records_csv_path = mlrun.get_sample_path('data/Taxi/yellow_tripdata_2019-01_subset.csv')
zones_csv_path = mlrun.get_sample_path('data/Taxi/taxi_zones.csv')

In [17]:
project.run_function(function='taxi',name='fetch_data',handler='fetch_data',
                               inputs={'taxi_records_csv_path': taxi_records_csv_path,
                                       'zones_csv_path': zones_csv_path},local=True).with_secrets("kubernetes",['AWS_ACCESS_KEY_ID','AWS_SECRET_ACCESS_KEY'])

> 2022-05-25 14:35:21,393 [info] starting run fetch_data uid=6261d6800c2f43e8901077d621ab9a0f DB=http://mlrun-api:8080
> 2022-05-25 14:35:21,582 [info] Reading taxi records data from https://s3.wasabisys.com/iguazio/data/Taxi/yellow_tripdata_2019-01_subset.csv
> 2022-05-25 14:35:26,342 [info] Reading zones data from https://s3.wasabisys.com/iguazio/data/Taxi/taxi_zones.csv
> 2022-05-25 14:35:26,488 [info] Saving datasets to v3io:///projects/nyc-remote-target-gilad/artifacts ...


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
nyc-remote-target-gilad,...21ab9a0f,0,May 25 14:35:21,completed,fetch_data,v3io_user=Giladkind=owner=Giladhost=jupyter-gilad-b89dbb79b-c8knv,taxi_records_csv_pathzones_csv_path,,,nyc-taxi-datasetzones-dataset


> 2022-05-25 14:35:34,134 [info] run executed, status=completed


#### This example shows how to deploy function without auto_mount

In [24]:
project.build_function('taxi')

> 2022-05-25 14:36:26,379 [info] Started building image: .mlrun/func-nyc-remote-target-gilad-taxi:latest
INFO[0000] Retrieving image manifest datanode-registry.iguazio-platform.app.cust-cs-3-2-3.iguazio-cd2.com:80/quay.io/mlrun/mlrun:1.1.0-rc2 
INFO[0000] Retrieving image datanode-registry.iguazio-platform.app.cust-cs-3-2-3.iguazio-cd2.com:80/quay.io/mlrun/mlrun:1.1.0-rc2 from registry datanode-registry.iguazio-platform.app.cust-cs-3-2-3.iguazio-cd2.com:80 
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Retrieving image manifest datanode-registry.iguazio-platform.app.cust-cs-3-2-3.iguazio-cd2.com:80/quay.io/mlrun/mlrun:1.1.0-rc2 
INFO[0000] Returning cached image manifest              
INFO[0000] Executing 0 build triggers                   
INFO[0000] Unpacking rootfs as cmd RUN python -m pip install lightgbm shapely requires it. 
INFO[0019] RUN python -m pip install lightgbm shapely   
INFO[0019] Taking snapshot of full filesystem...        
INFO[0046] cmd: /bin/

BuildStatus(ready=True, outputs={'image': '.mlrun/func-nyc-remote-target-gilad-taxi:latest'})

#### Deploy function with auto_mount - same way for all MLRun function types

In [45]:
taxi_func = project.get_function('taxi')

In [46]:
taxi_func.apply(mlrun.auto_mount())
taxi_func.deploy()

> 2022-05-25 14:55:23,774 [info] Started building image: .mlrun/func-nyc-remote-target-gilad-taxi:latest
INFO[0000] Retrieving image manifest datanode-registry.iguazio-platform.app.cust-cs-3-2-3.iguazio-cd2.com:80/quay.io/mlrun/mlrun:1.1.0-rc2 
INFO[0000] Retrieving image datanode-registry.iguazio-platform.app.cust-cs-3-2-3.iguazio-cd2.com:80/quay.io/mlrun/mlrun:1.1.0-rc2 from registry datanode-registry.iguazio-platform.app.cust-cs-3-2-3.iguazio-cd2.com:80 
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Retrieving image manifest datanode-registry.iguazio-platform.app.cust-cs-3-2-3.iguazio-cd2.com:80/quay.io/mlrun/mlrun:1.1.0-rc2 
INFO[0000] Returning cached image manifest              
INFO[0000] Executing 0 build triggers                   
INFO[0000] Unpacking rootfs as cmd RUN python -m pip install lightgbm shapely requires it. 
INFO[0019] RUN python -m pip install lightgbm shapely   
INFO[0019] Taking snapshot of full filesystem...        
INFO[0048] cmd: /bin/

True

In [27]:
project.run_function(function='taxi',name='fetch_data',handler='fetch_data',
                               inputs={'taxi_records_csv_path': taxi_records_csv_path,
                                       'zones_csv_path': zones_csv_path},local=False).with_secrets("kubernetes",['AWS_ACCESS_KEY_ID','AWS_SECRET_ACCESS_KEY'])

> 2022-05-25 14:38:53,494 [info] starting run fetch_data uid=c18c9bf33f024b69bc55cabec27d4a28 DB=http://mlrun-api:8080
> 2022-05-25 14:38:53,651 [info] Job is running in the background, pod: fetch-data-bwj7x
> 2022-05-25 14:38:57,693 [info] Reading taxi records data from https://s3.wasabisys.com/iguazio/data/Taxi/yellow_tripdata_2019-01_subset.csv
> 2022-05-25 14:39:02,865 [info] Reading zones data from https://s3.wasabisys.com/iguazio/data/Taxi/taxi_zones.csv
> 2022-05-25 14:39:03,093 [info] Saving datasets to v3io:///projects/nyc-remote-target-gilad/artifacts ...
> 2022-05-25 14:39:10,866 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
nyc-remote-target-gilad,...c27d4a28,0,May 25 14:38:57,completed,fetch_data,v3io_user=Giladkind=jobowner=Giladmlrun/client_version=1.1.0-rc2host=fetch-data-bwj7x,taxi_records_csv_pathzones_csv_path,,,nyc-taxi-datasetzones-dataset


> 2022-05-25 14:39:12,164 [info] run executed, status=completed


### Nuclio function

In [34]:
project.deploy_function('last_drive')

> 2022-05-25 14:45:49,368 [info] Starting remote function deploy
2022-05-25 14:45:49  (info) Deploying function
2022-05-25 14:45:49  (info) Building
2022-05-25 14:45:49  (info) Staging files and preparing base images
2022-05-25 14:45:49  (info) Building processor image
2022-05-25 14:45:56  (info) Build complete
2022-05-25 14:46:01  (info) Function deploy complete
> 2022-05-25 14:46:02,478 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-nyc-remote-target-gilad-last-drive.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['nyc-remote-target-gilad-last-drive-nyc-remote-target-gilad.default-tenant.app.cust-cs-3-2-3.iguazio-cd2.com/']}


DeployStatus(state=ready, outputs={'endpoint': 'http://nyc-remote-target-gilad-last-drive-nyc-remote-target-gilad.default-tenant.app.cust-cs-3-2-3.iguazio-cd2.com/', 'name': 'nyc-remote-target-gilad-last-drive'})

In [36]:
project.get_function('last_drive').invoke(path='nuclio-nyc-remote-target-gilad-last-drive.default-tenant.svc.cluster.local:8080')

> 2022-05-25 14:47:37,258 [info] invoking function: {'method': 'GET', 'path': 'http://nuclio-nyc-remote-target-gilad-last-drive.default-tenant.svc.cluster.local:8080/nuclio-nyc-remote-target-gilad-last-drive.default-tenant.svc.cluster.local:8080'}


{'hour': [4], 'day': [6], 'month': [1], 'year': [2019]}

### Serving function

In [37]:
serving = project.get_function("model-serving")
server = serving.to_mock_server()


> 2022-05-25 14:49:09,764 [info] model taxi-serving was loaded
> 2022-05-25 14:49:09,765 [info] Loaded ['taxi-serving']


In [41]:
my_data = '''{"inputs":[[1,0.80,0.711950,-1.291073,0.712059,1.290988,13,1,1,1,2019,47.274013,40.386065,16.975747,26.587155,18.925788]]}'''
resp = server.test('/v2/models/taxi-serving/predict', my_data)
resp

{'id': 'e1fad69dbc9d4c95aa1361f55f952d66',
 'model_name': 'taxi-serving',
 'outputs': [9.523029768974151]}

In [42]:
serving = project.get_function("model-serving")

In [43]:
serving.deploy()

> 2022-05-25 14:50:45,587 [info] Starting remote function deploy
2022-05-25 14:50:45  (info) Deploying function
2022-05-25 14:50:45  (info) Building
2022-05-25 14:50:45  (info) Staging files and preparing base images
2022-05-25 14:50:45  (info) Building processor image
2022-05-25 14:50:47  (info) Build complete
2022-05-25 14:50:53  (info) Function deploy complete
> 2022-05-25 14:50:53,378 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-nyc-remote-target-gilad-model-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['nyc-remote-target-gilad-model-serving-nyc-remote-target-gilad.default-tenant.app.cust-cs-3-2-3.iguazio-cd2.com/']}


'http://nyc-remote-target-gilad-model-serving-nyc-remote-target-gilad.default-tenant.app.cust-cs-3-2-3.iguazio-cd2.com/'

In [44]:
my_data = '''{"inputs":[[1,0.80,0.711950,-1.291073,0.712059,1.290988,13,1,1,1,2019,47.274013,40.386065,16.975747,26.587155,18.925788]]}'''
serving.invoke('/v2/models/taxi-serving/predict', my_data)

> 2022-05-25 14:51:05,008 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-nyc-remote-target-gilad-model-serving.default-tenant.svc.cluster.local:8080/v2/models/taxi-serving/predict'}


{'id': 'cdcec970-6578-4707-a375-8d025827cc0d',
 'model_name': 'taxi-serving',
 'outputs': [9.523029768974151]}